# NeXus classes

## Overview

NeXus provides a substantial number of [base class definitions](https://manual.nexusformat.org/classes/base_classes/index.html#base-class-definitions).
At this point ScippNexus supports only a very limited number of these.
Furthermore, not all features of each class definition are implemented.
The class of a group is read from the group's `'NX_class'` attribute.

The following table gives an overview of the most important supported classes.
There are roughly two categories of classes, those that contain data and can be read as a `scipp.DataArray` (for example NXdata), and those that mostly serve as groups for nested classes (for example NXentry containing NXdata):

NeXus class | read as | comment | NeXus specification
:--- |:--- |:--- |:---
[NXdata](../generated/classes/scippnexus.NXdata.rst) | scipp.DataArray | [example below](#NXdata) | [link](https://manual.nexusformat.org/classes/base_classes/NXdata.html)
[NXdetector](../generated/classes/scippnexus.NXdetector.rst) | scipp.DataGroup wrapping scipp.DataArray | [example below](#NXdetector) | [link](https://manual.nexusformat.org/classes/base_classes/NXdetector.html)
[NXdisk_chopper](../generated/classes/scippnexus.NXdisk_chopper.rst) | scipp.DataGroup || [link](https://manual.nexusformat.org/classes/base_classes/NXdisk_chopper.html)
[NXentry](../generated/classes/scippnexus.NXentry.rst) | scipp.DataGroup | [generic group-like](#Base-class:-NXobject) | [link](https://manual.nexusformat.org/classes/base_classes/NXentry.html)
[NXevent_data](../generated/classes/scippnexus.NXevent_data.rst) | scipp.DataArray | [example below](#NXevent_data) | [link](https://manual.nexusformat.org/classes/base_classes/NXevent_data.html)
[NXinstrument](../generated/classes/scippnexus.NXinstrument.rst) | scipp.DataGroup | [generic group-like](#Base-class:-NXobject) | [link](https://manual.nexusformat.org/classes/base_classes/NXinstrument.html)
[NXlog](../generated/classes/scippnexus.NXlog.rst) | scipp.DataArray | [example below](#NXlog) | [link](https://manual.nexusformat.org/classes/base_classes/NXlog.html)
[NXmonitor](../generated/classes/scippnexus.NXmonitor.rst) | scipp.DataGroup wrapping scipp.DataArray | [example below](#NXmonitor) | [link](https://manual.nexusformat.org/classes/base_classes/NXmonitor.html)
[NXroot](../generated/classes/scippnexus.NXroot.rst) | scipp.DataGroup | [generic group-like](#Base-class:-NXobject) | [link](https://manual.nexusformat.org/classes/base_classes/NXroot.html)
[NXsample](../generated/classes/scippnexus.NXsample.rst) | scipp.DataGroup || [link](https://manual.nexusformat.org/classes/base_classes/NXsample.html)
[NXsource](../generated/classes/scippnexus.NXsource.rst) | scipp.DataGroup || [link](https://manual.nexusformat.org/classes/base_classes/NXsource.html)
[NXtransformations](../generated/classes/scippnexus.NXtransformations.rst) | scipp.DataGroup | [generic group-like](#Base-class:-NXobject) | [link](https://manual.nexusformat.org/classes/base_classes/NXtransformations.html)

Any class that is not explicitly supported will be loaded as a `scipp.DataGroup` as well.

For the examples below we use a file from the ScippNexus sample data:

In [ ]:
from scippnexus import data
import scippnexus as snx

filename = data.get_path('PG3_4844_event.nxs')
f = snx.File(filename)

## Base class: NXobject

Base of all other NeXus classes.
Provides a generic group-like interface.
That is, this is equivalent to a dictionary of fields and/or other groups.

NeXus classes that group other information but cannot be read as a data array provide this interface.
Such groups will be loaded as `scipp.DataGroup` when loaded:

In [ ]:
tree = f['entry/instrument'][()]
tree

## NXdata

Provides multi-dimensional labeled data.
See the NeXus format [NXdata base class definition](https://manual.nexusformat.org/classes/base_classes/NXdata.html) for details.
Can be read as a data array using positional indexing.

Example:

In [ ]:
data = f['entry/bank103']
data

In [ ]:
data['x_pixel_offset', :10]

## NXdetector

Provides data for a detector.
See the NeXus format [NXdetector base class definition](https://manual.nexusformat.org/classes/base_classes/NXdetector.html) for details.
`NXdetector` contains data and coords similar to `NXdata` as well as additional fields that are not readily inserted into a `scipp.DataArray`.
Therefore, `NXdetector` can be read as a `scipp.DataGroup` wrapping a `scipp.DataArray` using positional indexing.
The "signal" field and associated coordinates are combined into a `scipp.DataArray` and the remaining fields are added as fields to the `scipp.DataGroup`.
In the output, the `scipp.DataArray` has the same name as the "signal" dataset in the NeXus file.
The underlying data may be dense data or event data.

Example:

In [ ]:
detector = f['entry/instrument/bank102']
detector

In [ ]:
det = detector[...]
det

In this example both dense data ("data_x") and event data ("events") are present:

In [ ]:
det['data_x_y']

In [ ]:
det['events']

If the underlying data is event data, the underlying event data can be selected using the special `event_time_zero` dimension.
This dimension is present in the underlying `NXevent_data` group, but not preserved after loading and binning by pixels due to the prohibitive size.
For example, we can select the first 1000 pulses and load data for all pixels:

In [ ]:
detector['event_time_zero', :1000]

<div class="alert alert-info">
    <b>Note:</b>

Selecting a range of events allows for loading only a potentially very small section of the underlying event data and can thus be very fast.

In contrast, e.g., selecting a small range of pixels in presence of underlying event data is *not* fast, since the events for all pixels are stored in the order as they arrive in the acquisition system and the entire [NXevent_data](#NXevent_data) group must be loaded.

</div>

## NXevent_data

Provides event data in raw format as produced by the acquisition system, i.e., not grouped into detector pixels.
See the NeXus format [NXevent_data base class definition](https://manual.nexusformat.org/classes/base_classes/NXevent_data.html) for details.
Can be read as a data array using slicing syntax.

Example:

In [ ]:
event_data = f['entry/bank102_events']
event_data[...]

In some cases the event data fields may be contained directly within an [NXdetector](#NXdetector).
The event data can also be accessed from there:

In [ ]:
f['entry/instrument/bank102']['events'][...]

## NXlog

Provides a  time-series log.
See the NeXus format [NXlog base class definition](https://manual.nexusformat.org/classes/base_classes/NXlog.html) for details.
Can be read as a data array using positional indexing.

Example:

In [ ]:
proton_charge = f['/entry/DASlogs/proton_charge']
proton_charge

In [ ]:
proton_charge[...]

## NXmonitor

Provides data for a beam monitor.
See the NeXus format [NXmonitor base class definition](https://manual.nexusformat.org/classes/base_classes/NXmonitor.html) for details.
Can be read as a `scipp.DataGroup` holding a `scipp.DataArray` using positional indexing, similar to `NXdetector`.

Example:

In [ ]:
monitor = f['entry/monitor1']
mon = monitor[...]
mon

In [ ]:
mon['data']